* _Note_: The plot previews do not work in Github, but work [here](http://nbviewer.jupyter.org/github/SkarletSkay/DE_Assignment/blob/master/DE%20assignment%20%285%29.ipynb)

In [1]:
%matplotlib inline
import numpy  as np
import matplotlib
import matplotlib.pyplot as plt

<h2>Functions:</h2>

* **Differential Equation**


* **Differential Equation Solution**
    
    $ \frac{-3e^{2x}}{e^{3x} - 6e^{2} - e^{3}} $
    
    and this function is discontinious at the region where x=~1.38
    
    So it should be considered in our numerical methods, because otherwise they will not work correct.
    
    I take the region (dis1, dis2), where methods do not work because of overflow in python.
    
    Because point (1, 0.5) on the left side, therefore after discontinuity there is not any value of y to continue to calculate by numerical methods.
    
    Actually numerical methods cannot use the exact soultion but in this case let's consider it is possible, because otherwise we cannot calculate the right side of the plot. 
    
    yd is y(x) calculated using the exact solution, where x is the first point which consider in the plot after the discontinuity.
    
    x0 $= x_0$
    
    y0 $= y(x_0)$
    
    X is the right bound of the interval $ [x_0; X] $
    
    (dis1, dis2) is bounds of discontinuity region
    
    So the functions gets (x0, y0, dis1, dis2, yd, X)
    

* **Euler's method**

    $ y_{n+1} = y_{n} + h * f(x_{n}, y_{n}) $
    
    
* **Improved Euler's method**
    
    $ k_{1n} = f(x_{n}, y_{n}) $
    
    $ k_{2n} = f(x_{n} + h, y_{n} + hk_{1n}) $
    
    $ y_{n+1} = y_{n} + \frac{h}{2} * (k_{1n}+k_{2n}) $
    
    
* **Runge-Kutta method**

    $ k_{1n} = f(x_{n}, y_{n}) $
    
    $ k_{2n} = f(x_{n} + \frac{h}{2}, y_{n} + \frac{h}{2} * k_{1n}) $
    
    $ k_{3n} = f(x_{n} + \frac{h}{2}, y_{n} + \frac{h}{2} * k_{2n}) $
    
    $ k_{4n} = f(x_{n} + h, y_{n} + hk_{3n}) $
    
    $ y_{n+1} = y_{n} + \frac{h}{6} * (k_{1n}+2k_{2n}+2k_{3n}+k_{4n}) $
    

In [2]:
def f(x, y):
    return y*y*np.exp(x) + 2 * y

In [3]:
def exact(x):
    a = (np.exp(3*x) - (np.exp(2)*6) - np.exp(3))
    res = (-3*np.exp(2*x))/a
    return res

exact(np.array([1])) # y(1) = 0.5

array([0.5])

In [4]:
GRID_SIZE = 100
X = 7
x0 = 1
y0 = 0.5

h = (X - x0)/GRID_SIZE
x = np.arange(x0, X, h) #Let's have the separate list of x for easier calculation of errors

In [5]:
def euler(y0, dis1, dis2, yd, X):
    ys = []
    y = y0
    for t in x :
        if t > dis1 and t < dis2: # Set Not a Number for discountinuity region
            ys.append(np.nan)
            y = yd
        else:
            y_n  = y + h*f(t,y)
            ys.append(y)
            y = y_n
    return ys 
    

In [6]:
def impr_euler(y0,dis1, dis2, yd, X):
    ys = []
    y = y0
    for t in x :
        if t > dis1 and t<dis2: # Set Not a Number for discountinuity region
            ys.append(np.nan)
            y = yd
        else:
            k1 = f(t,y)
            k2 = f(t+h,y + h*k1)
            y_n  = y + (h/2)*(k1+k2)
            ys.append(y)
            y = y_n
    return ys 

In [7]:
def runge_kutta(y0,dis1, dis2, yd, X):
    ys =[]
    y = y0
    for t in x :
        if t > dis1 and t<dis2: # Set Not a Number for discountinuity region
            ys.append(np.nan)
            y = yd
        else:
            k1 = f(t,y)
            k2 = f(t+h/2,y + h*k1/2)
            k3 = f(t+h/2,y + h*k2/2)
            k4 = f(t+h,y + h*k3)
            y_n  = y + (h/6)*(k1+2*k2 + 2*k3 + k4)
            ys.append(y)
            y = y_n
    return ys 

In [8]:
y = exact(x);
#Using exact solution find the discontinuity. Let's consider it is the difference between Xn+1 - Xn >= 5 
pos = np.where(np.diff(y)>=5)[0]

#Find bounds of discontinious region
if len(pos) > 0: 
    dis1 = x[pos[0] - 1]
    dis2 = x[pos[-1] + 1]
    yd = y[pos[-1] + 1]
else:
    dis1 = -1
    dis2 = -1
    yd = -1
    
#Calculate using the numerical methods
eul = euler(y0,dis1,dis2, yd,X)
i_eul = impr_euler(y0,dis1,dis2, yd,X)
run_k = runge_kutta(y0,dis1,dis2, yd,X)

# Set Not a Number for discountinuity region 
if(len(pos)>0):
    for i in range(pos[0],pos[-1],1):
        y[i] = np.nan



## Plots

In [9]:
import plotly.plotly as plotl
import plotly.graph_objs as go

#Plotting
exact_trace = go.Scatter(
    name = "Exact",
    x = x,
    y = y
)

euler_trace = go.Scatter(
    name = "Euler's",
    x = x,
    y = eul
)

i_euler_trace = go.Scatter(
    name = "Improved Euler's",
    x = x,
    y = i_eul
)

runge_kutta_trace = go.Scatter(
    name = "Runge-Kutta",
    x = x,
    y = run_k
)
#Plot axis and title
layout = dict(title = 'Solution Before Discontinuity',
              xaxis = dict(title = 'X axis'),
              yaxis = dict(title = 'Y axis'),
              )

data = [exact_trace, euler_trace, i_euler_trace, runge_kutta_trace]
fig = dict(data=data, layout=layout)
plotl.iplot(fig, filename='basic-line1') 

D:\Programming\Python\DifferentialEquations\venv\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Only Left Side
Actually we know that there is not true numerical method used for the right side, let's check only left side now.
Error increase every step.

In [10]:
X = 1.38
h = (X - x0)/GRID_SIZE
x = np.arange(x0, X, h)
y = exact(x);
    
dis1 = -1
dis2 = -1
yd = -1
    
#Calculate using the numerical methods
eul = euler(y0,dis1,dis2, yd,X)
i_eul = impr_euler(y0,dis1,dis2, yd,X)
run_k = runge_kutta(y0,dis1,dis2, yd,X)





In [11]:
#Plotting
exact_trace = go.Scatter(
    name = "Exact",
    x = x,
    y = y
)

euler_trace = go.Scatter(
    name = "Euler's",
    x = x,
    y = eul
)

i_euler_trace = go.Scatter(
    name = "Improved Euler's",
    x = x,
    y = i_eul
)

runge_kutta_trace = go.Scatter(
    name = "Runge-Kutta",
    x = x,
    y = run_k
)
#Plot axis and title
layout = dict(title = 'Solution Before Discontinuity',
              xaxis = dict(title = 'X axis'),
              yaxis = dict(title = 'Y axis'),
              )

data = [exact_trace, euler_trace, i_euler_trace, runge_kutta_trace]
fig = dict(data=data, layout=layout)
plotl.iplot(fig, filename='basic-line2') 

## Global Errors
Error decrease when grid size increase, because we have smaller steps

In [12]:
eul_errors = []
i_eul_errors = []
runge_kutta_errors = []
for j in range(100,500,10):
    X = 1.38
    h = (X - x0)/j
    x = np.arange(x0, X, h)
    y = exact(x);
    
    dis1 = -1
    dis2 = -1
    yd = -1
    
    #Calculate using the numerical methods
    eul = euler(y0,dis1,dis2, yd,X)
    i_eul = impr_euler(y0,dis1,dis2, yd,X)
    run_k = runge_kutta(y0,dis1,dis2, yd,X)

    ex_v = dict((x, y) for x, y in zip(x,y))
    eul_v = dict((x, y) for x, y in zip(x,eul))
    i_eul_v = dict((x, y) for x, y in zip(x, i_eul))
    run_k_v = dict((x, y) for x, y in zip(x, run_k))

    eul_t_err = 0
    i_eul_t_err = 0
    run_k_t_err = 0
    #Calculate error
    for t in x:
         
        temp = np.abs(ex_v[t]-eul_v[t])
        eul_t_err = eul_t_err + temp
                
        temp = np.abs(ex_v[t]-i_eul_v[t])
        i_eul_t_err = i_eul_t_err+temp
                
        temp = np.abs(ex_v[t]-run_k_v[t])
        run_k_t_err = run_k_t_err+temp
    
    #Add total errors of this grid size to the lists
    eul_errors.append(eul_t_err/j)
    i_eul_errors.append(i_eul_t_err/j)
    runge_kutta_errors.append(run_k_t_err/j)


In [13]:
#Plotting
euler_total_err = go.Scatter(
    name = "Euler's",
    x = np.arange(100,500,10),
    y = eul_errors
)

impr_euler_total_err = go.Scatter(
    name = "Improved Euler's",
    x = np.arange(100,500,10),
    y = i_eul_errors
)

runge_kutta_trace_err = go.Scatter(
    name = "Runge-Kutta",
    x = np.arange(100,500,10),
    y = runge_kutta_errors
)

#Plot axis and title
layout = dict(title = 'Global Errors Graph',
              xaxis = dict(title = 'Gride Size'),
              yaxis = dict(title = 'Global Error'),
              )


data = [euler_total_err, impr_euler_total_err,runge_kutta_trace_err]
fig = dict(data=data, layout=layout)
plotl.iplot(fig, filename='basic-line3') 